# 07 - Vertex AI > Features - Feature Store

This is a demonstration of [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore/overview). A feature store is a central repository for organizing, storing, and retrieving features.  This is a fully managed service that scales the underlying compute and storage resources.  The feature store becomes a central location for serving features for training and prediction with low-latency. It stores feature values at points-in-time:

-  Point-in-time lookups for retrieving features for model training. Retrieve feature values prior to a prediction to prevent data leakage.
-  Manage training-serving skew

**Prerequisites:**

-  01 - BigQuery - Table Data Source
-  Any of 02-05 That Deploy A Model To An Endpoint
   -  Used to demonstrate online predictions with feature store serving features

**Overview:**

-  Create a Feature Store
-  Define an entity type
-  Define features for and entity type
   -  For this demonstration I use metadata from a BigQuery table to define features
      -  project.table.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
-  Search Features
   -  Using FeaturestoreServiceClient.search_features()
-  Import feature values
   -  Using a BigQuery table as the source data
-  Serve feature values
   -  For one entity_Id
   -  For multiple entity_id values
   -  Batch Feature request
-  Use online feature serving as input to online prediction with Vertex AI Endpoint

**Resources:**

-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
   -  Currently using the [v1beta1 services](https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/services.html)
-  [Feature Store Overview](https://cloud.google.com/vertex-ai/docs/featurestore/overview)
-  [Data Model and Concepts](https://cloud.google.com/vertex-ai/docs/featurestore/concepts)
-  [Best Practices](https://cloud.google.com/vertex-ai/docs/featurestore/best-practices) including info on composite entity types

**Related Training:**

-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-07.png">

---
## Setup

inputs:

In [37]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '07'

ENTITYTYPE_ID = 'drawing'

packages:

In [96]:
from google.cloud.aiplatform_v1beta1 import (FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient, types)
from google.cloud import aiplatform

from google.protobuf.duration_pb2 import Duration
from google.protobuf.timestamp_pb2 import Timestamp
from google.protobuf.field_mask_pb2 import FieldMask

from google.cloud import bigquery
from google.cloud.aiplatform_v1beta1 import (PredictionServiceClient, EndpointServiceClient)
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [74]:
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

clients = {}
clients['fs'] = FeaturestoreServiceClient(client_options = client_options)
clients['fs_olserve'] = FeaturestoreOnlineServingServiceClient(client_options = client_options)

clients['bq'] = bigquery.Client()

aiplatform.init(project=PROJECT_ID, location=REGION)

parameters:

In [42]:
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"
DIR = f"temp/{NOTEBOOK}"

environment:

In [43]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Feature Store Data model
Feature Store organizes data with the following 3 important hierarchical concepts:

Featurestore -> EntityType -> Feature

- **Featurestore**: the place to store your features
    - **EntityType**: under a Featurestore, an EntityType describes an object to be modeled, real one or virtual one.
        - **Feature**: under an EntityType, a feature describes an attribute of the EntityType

For the digits data used in these examples, the feature store will be called digits_featurestore.  The store has 1 entity type: images.  The features will be the pixels and the target values.

---
## Create Feature Store

In [10]:
FEATURESTORE_ID = DATANAME

In [11]:
featurestore_lro = clients['fs'].create_featurestore(
    types.featurestore_service.CreateFeaturestoreRequest(
        parent = PARENT,
        featurestore_id = FEATURESTORE_ID,
        featurestore=types.featurestore.Featurestore(
            display_name=f"Notebook {NOTEBOOK} demonstration of Vertex AI Features (feature store) using {DATANAME} data",
            online_serving_config=types.featurestore.Featurestore.OnlineServingConfig(
                fixed_node_count=2
            ),
        ),
    )
)

In [12]:
featurestore_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits"

Use `get_featurestore` to see details of specified feature store:

In [13]:
clients['fs'].get_featurestore(name=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID))

name: "projects/691911073727/locations/us-central1/featurestores/digits"
create_time {
  seconds: 1631808488
  nanos: 193234000
}
update_time {
  seconds: 1631808488
  nanos: 502681000
}
etag: "AMEw9yNDxF_abALl9D9yDmEc9TOeOoFaCwKQEoEh-Zlm_2UNiiZXa2FG65VrSk8aDEqh"
online_serving_config {
  fixed_node_count: 2
}
state: STABLE

Use `list_featurestores` to see details of all feature stores:

In [14]:
clients['fs'].list_featurestores(parent=PARENT)

ListFeaturestoresPager<featurestores {
  name: "projects/691911073727/locations/us-central1/featurestores/digits"
  create_time {
    seconds: 1631808488
    nanos: 193234000
  }
  update_time {
    seconds: 1631808488
    nanos: 502681000
  }
  etag: "AMEw9yNz4KutuBpX5iothDdrViHBUReWh-ud3YLPn5TIz4VdW-nzDTex7NhAM_JDnmKW"
  online_serving_config {
    fixed_node_count: 2
  }
  state: STABLE
}
>

---
## Create Entity Type

In [17]:
entitytype_lro = clients['fs'].create_entity_type(
    types.featurestore_service.CreateEntityTypeRequest(
        parent=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id = ENTITYTYPE_ID,
        entity_type=types.entity_type.EntityType(
            description=f"Entity: {ENTITYTYPE_ID}, for data: {DATANAME}",
            monitoring_config=types.featurestore_monitoring.FeaturestoreMonitoringConfig(
                snapshot_analysis=types.featurestore_monitoring.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=900),  # 15 minutes
                ),
            ),
        ),
    )
)

In [18]:
entitytype_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing"
etag: "AMEw9yOEUHP5x4BBKSDXBEpYikMafo8LHHBzMT0Gnm8EnDQCFqyX"

Use `list_entity_types` to see details of all entity types:

In [19]:
clients['fs'].list_entity_types(parent = f"{PARENT}/featurestores/{FEATURESTORE_ID}")

ListEntityTypesPager<entity_types {
  name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing"
  description: "Entity drawing for  data digits"
  create_time {
    seconds: 1631812110
    nanos: 831664000
  }
  update_time {
    seconds: 1631812110
    nanos: 831664000
  }
  etag: "AMEw9yNKJa4P5Pkfgf-N5epF4pLTpAL61bICdz8w1PXmG5D7PhmJfsdmmwdUH-9o_Rip"
  monitoring_config {
    snapshot_analysis {
      monitoring_interval {
        seconds: 86400
      }
    }
  }
}
>

---
## Create Features

Get the schema of the data source for new features:

In [20]:
%%bigquery schema
SELECT * 
FROM `statmike-mlops.digits.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'digits'

Downloading: 100%|██████████| 66/66 [00:00<00:00, 67.41rows/s] 


In [21]:
schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,statmike-mlops,digits,digits,p0,1,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
1,statmike-mlops,digits,digits,p1,2,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
2,statmike-mlops,digits,digits,p2,3,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
3,statmike-mlops,digits,digits,p3,4,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
4,statmike-mlops,digits,digits,p4,5,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,statmike-mlops,digits,digits,p61,62,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
62,statmike-mlops,digits,digits,p62,63,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
63,statmike-mlops,digits,digits,p63,64,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
64,statmike-mlops,digits,digits,target,65,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN


Prepare a request for `batch_create_features`:
- specification for the features, data type and descriptions ....

In [23]:
REQUESTS = []
for i in range(schema.shape[0]):
    
    if schema['data_type'][i] == 'STRING': value_type = types.feature.Feature.ValueType.STRING
    elif schema['data_type'][i] == 'INT64': value_type = types.feature.Feature.ValueType.INT64
    elif schema['data_type'][i] == 'FLOAT64': value_type = types.feature.Feature.ValueType.DOUBLE
    
    description = f"Column named {schema['column_name'][i]} from BQ Table {PROJECT_ID}.{DATANAME}.{DATANAME}"
    
    REQUESTS.append(
        types.featurestore_service.CreateFeatureRequest(
            feature=types.feature.Feature(
                value_type = value_type,
                description = description,
                # optional, monitoring_config here as override, otherwise it inherits from entity_type
            ),
            feature_id = schema['column_name'][i].lower(),
        )    
    )

In [24]:
batchfeatures = clients['fs'].batch_create_features(
    parent = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    requests = REQUESTS,
)

In [26]:
#list(item.name for item in batchfeatures.result().features)

---
## Search Features
Search goes across all Feature Stores and Entity Types.

Also, use the list_features function to list all.

In [27]:
# return the first feature:
list(clients['fs'].search_features(location = PARENT))[0]

name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing/features/p0"
description: "Column named p0 from BQ Table statmike-mlops.digits.digits"
create_time {
  seconds: 1631812646
  nanos: 231977000
}
update_time {
  seconds: 1631812646
  nanos: 231977000
}

In [28]:
# find all features with INT64 value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location = PARENT, query = "value_type=INT64")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing/features/target"
 description: "Column named target from BQ Table statmike-mlops.digits.digits"
 create_time {
   seconds: 1631812646
   nanos: 328113000
 }
 update_time {
   seconds: 1631812646
   nanos: 328113000
 }]

In [30]:
# find all features of the form p*9 with DOUBLE value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location = PARENT, query = "feature_id:p*9 AND value_type=DOUBLE")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing/features/p19"
 description: "Column named p19 from BQ Table statmike-mlops.digits.digits"
 create_time {
   seconds: 1631812646
   nanos: 256765000
 }
 update_time {
   seconds: 1631812646
   nanos: 256765000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing/features/p29"
 description: "Column named p29 from BQ Table statmike-mlops.digits.digits"
 create_time {
   seconds: 1631812646
   nanos: 269066000
 }
 update_time {
   seconds: 1631812646
   nanos: 269066000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTypes/drawing/features/p39"
 description: "Column named p39 from BQ Table statmike-mlops.digits.digits"
 create_time {
   seconds: 1631812646
   nanos: 284866000
 }
 update_time {
   seconds: 1631812646
   nanos: 284866000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits/entityTyp

---
## Import Feature Values
- BigQuery (THIS DEMO)
- Avro
- CSV

Prepare a source table with timestamp (update_time) and unique id's for each entity

In [31]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops.digits.digits_featurestore_import` AS
WITH A AS (SELECT GENERATE_UUID() unique_id, target_OE as target_oe, CAST(FLOOR(10*RAND()) AS INT64) daytrick, * EXCEPT(target_OE) FROM `statmike-mlops.digits.digits`)
SELECT * EXCEPT(daytrick), DATE_SUB(CURRENT_TIMESTAMP, INTERVAL daytrick DAY) AS update_time FROM A

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1598.85query/s]                        


""


Create Feature specification for each feature in the input source:

In [32]:
FEATURE_SPECS = []
for i in range(schema.shape[0]):
    FEATURE_SPECS.append(types.featurestore_service.ImportFeatureValuesRequest.FeatureSpec(id = schema['column_name'][i].lower()))

In [33]:
import_request = types.featurestore_service.ImportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    bigquery_source = types.BigQuerySource(input_uri = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_featurestore_import'),
    feature_time_field = "update_time",
    feature_time = Timestamp().GetCurrentTime(),
    entity_id_field = "unique_id",
    feature_specs = FEATURE_SPECS,
    worker_count = 4,
)

In [34]:
importjob = clients['fs'].import_feature_values(import_request)

In [35]:
importjob.result()

imported_entity_count: 1797
imported_feature_value_count: 118602

---
## Prediction with Feature Store for Serving Features

### Entity Id's
Retrieve a list of entity id's from the source BigQuery table.  These are in the column `unique_id`.

In [53]:
unique_id = clients['bq'].query(query = f"SELECT unique_id FROM {PROJECT_ID}.{DATANAME}.{DATANAME}_featurestore_import LIMIT 5").to_dataframe()

In [54]:
unique_id['unique_id'][0]

'd2c55b92-c10e-4ef5-a61c-f7e21010a982'

### Data For Prediction: Single Entity Served by Vertex AI > Features (Feature Store)

In [55]:
feature_values = clients['fs_olserve'].read_feature_values(
    types.featurestore_online_service.ReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_id = unique_id['unique_id'][0],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [57]:
print(list(item.id for item in feature_values.header.feature_descriptors))

['p10', 'target', 'p42', 'p63', 'p9', 'p5', 'p7', 'p44', 'p39', 'p28', 'p17', 'p29', 'p36', 'p38', 'p50', 'p45', 'p13', 'p35', 'p49', 'p62', 'p25', 'p14', 'p41', 'p1', 'p61', 'p8', 'p47', 'p20', 'p52', 'p27', 'p33', 'p31', 'p2', 'p46', 'p12', 'p21', 'p0', 'p23', 'p43', 'p58', 'target_oe', 'p32', 'p54', 'p18', 'p48', 'p11', 'p40', 'p34', 'p37', 'p6', 'p55', 'p15', 'p60', 'p22', 'p4', 'p59', 'p56', 'p53', 'p3', 'p57', 'p26', 'p24', 'p16', 'p19', 'p51', 'p30']


In [58]:
print(list(item.value.double_value for item in feature_values.entity_view.data))

[9.0, 0.0, 16.0, 0.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 10.0, 0.0, 8.0, 11.0, 14.0, 12.0, 0.0, 0.0, 0.0, 6.0, 0.0, 3.0, 0.0, 6.0, 0.0, 0.0, 0.0, 16.0, 0.0, 7.0, 0.0, 1.0, 7.0, 16.0, 13.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 14.0, 0.0, 16.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 16.0, 3.0, 12.0, 11.0, 0.0, 16.0, 13.0, 0.0, 10.0, 0.0, 0.0, 6.0, 16.0, 6.0]


### Prepare Prediction Request

In [60]:
newob = {}
features = list(item.id for item in feature_values.header.feature_descriptors)
for e, f in enumerate(features):
    if f.startswith('p'):
        newob[f] = feature_values.entity_view.data[e].value.double_value

In [62]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Pick An Endpoint
A list index of [0] here retrieves the first endpoint in this project:

In [83]:
aiplatform.Endpoint.list()[0].display_name

'04a_digits_20210916111550'

In [91]:
endpoint = aiplatform.Endpoint(endpoint_name = aiplatform.Endpoint.list()[0].name)

### Get Predictions: Python Client

In [92]:
prediction = endpoint.predict(instances = instances, parameters = parameters)

In [93]:
prediction

Prediction(predictions=[[0.999998927, 1.82380066e-11, 6.84882536e-07, 6.92010627e-09, 2.78563981e-08, 9.37663316e-08, 8.60855423e-08, 6.19005958e-09, 1.76400889e-07, 2.36027162e-08]], deployed_model_id='1563391185488183296', explanations=None)

In [94]:
prediction.predictions[0]

[0.999998927,
 1.82380066e-11,
 6.84882536e-07,
 6.92010627e-09,
 2.78563981e-08,
 9.37663316e-08,
 8.60855423e-08,
 6.19005958e-09,
 1.76400889e-07,
 2.36027162e-08]

In [97]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [98]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [99]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999998927,
      1.82380066e-11,
      6.84882536e-07,
      6.92010627e-09,
      2.78563981e-08,
      9.37663316e-08,
      8.60855423e-08,
      6.19005958e-09,
      1.76400889e-07,
      2.36027162e-08
    ]
  ],
  "deployedModelId": "1563391185488183296"
}


### Get Predictions: gcloud (CLI)

In [100]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999998927, 1.82380066e-11, 6.84882536e-07, 6.92010627e-09, 2.78563981e-08, 9.37663316e-08, 8.60855423e-08, 6.19005958e-09, 1.76400889e-07, 2.36027162e-08]]


### Data For Prediction: Multiple Entities Served by Vertex AI > Features (Feature Store)

In [107]:
unique_id['unique_id']

0    d2c55b92-c10e-4ef5-a61c-f7e21010a982
1    a513340f-45a2-4186-ad78-88d0f84abab9
2    54ac5d3b-249d-4ac5-a8c0-bfe590bad3f2
3    bcb853a0-fe04-4af1-a930-bb2118a4aa34
4    bc12dad9-ade6-4180-b592-dec0512a0eaa
Name: unique_id, dtype: object

In [108]:
multi_feature_values = clients['fs_olserve'].streaming_read_feature_values(
    types.featurestore_online_service.StreamingReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_ids = unique_id['unique_id'],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [109]:
for i in multi_feature_values:
    print(i.entity_view.entity_id)
    print(list(item.value.double_value for item in i.entity_view.data))


[]
54ac5d3b-249d-4ac5-a8c0-bfe590bad3f2
[10.0, 0.0, 0.0, 14.0, 1.0, 12.0, 0.0, 0.0, 9.0, 0.0, 0.0, 16.0, 2.0, 8.0, 0.0, 14.0, 13.0, 0.0, 13.0, 0.0, 0.0, 5.0, 0.0, 0.0, 8.0, 0.0, 2.0, 15.0, 3.0, 10.0, 7.0, 12.0, 0.0, 0.0, 1.0, 3.0, 14.0, 5.0, 0.0, 15.0, 3.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 15.0, 0.0, 15.0, 0.0, 0.0, 15.0, 0.0, 0.0, 4.0, 7.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0]
a513340f-45a2-4186-ad78-88d0f84abab9
[10.0, 0.0, 0.0, 9.0, 3.0, 14.0, 0.0, 0.0, 5.0, 0.0, 0.0, 12.0, 2.0, 8.0, 0.0, 16.0, 11.0, 0.0, 12.0, 0.0, 0.0, 1.0, 0.0, 0.0, 12.0, 0.0, 8.0, 12.0, 4.0, 9.0, 6.0, 12.0, 0.0, 0.0, 1.0, 0.0, 13.0, 8.0, 0.0, 13.0, 6.0, 0.0, 3.0, 8.0, 0.0, 0.0, 0.0, 0.0, 12.0, 3.0, 14.0, 0.0, 16.0, 0.0, 0.0, 15.0, 0.0, 0.0, 4.0, 8.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0]
bc12dad9-ade6-4180-b592-dec0512a0eaa
[9.0, 0.0, 0.0, 5.0, 1.0, 9.0, 0.0, 0.0, 2.0, 0.0, 0.0, 13.0, 0.0, 5.0, 0.0, 5.0, 8.0, 0.0, 13.0, 0.0, 2.0, 0.0, 7.0, 0.0, 5.0, 0.0, 12.0, 14.0, 2.0, 10.0, 3.0, 8.0, 0.0, 13.0, 0.0, 0.0, 16.0,

### Data For Training: Batch (For training or large scale prediction)

In [155]:
# get current timestamp (protobuf3 is seconds since ephoch (1970))
timestamp = Timestamp()
timestamp.GetCurrentTime()

# adjust timestamp to 2 days ago: 60*60*24*4
newtimestamp = Timestamp(seconds = timestamp.seconds - 60*60*24*2, nanos = timestamp.nanos)

batch_request = types.featurestore_service.ExportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    snapshot_export = types.ExportFeatureValuesRequest.SnapshotExport(snapshot_time = Timestamp(seconds=newtimestamp.seconds)),
    destination = types.FeatureValueDestination(bigquery_destination = types.BigQueryDestination(output_uri = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_fs_training')),
    feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids = ['*']))
)

In [156]:
batchjob = clients['fs'].export_feature_values(batch_request)

In [157]:
batchjob.result()

By Adjusting the `snapshot_time` to 2 days ago, the batch_request creates a BigQuery table that has all the orginal rows, 1 per entity, but the features are null for 20% of the rows.  This is because the features were loaded with `feature_time_field = "update_time"` and `update_time` was set to a random day between today and 10 days ago.

In [159]:
%%bigquery
SELECT CASE WHEN p0 is not null then False ELSE True END as Null_Rows, count(*) as counts
FROM statmike-mlops.digits.digits_fs_training
GROUP BY Null_Rows

Downloading: 100%|██████████| 2/2 [00:00<00:00,  2.10rows/s]


,Null_Rows,counts
0,True,364
1,False,1433


---
## Remove Resources
see notebook "XX - Cleanup"